Imports

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import torch
from torch.utils.data import Dataset, DataLoader

sys.path.append('../../')
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from src.data.data_utils import *
from src.data.dataset import *

/u/ayang1/.conda/envs/CropTS/lib/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Data Variables

In [11]:
save_path = '/scratch/bbug/ayang1/datasets/lucas_dataset_large/' # where to save data
data_root = '/scratch/bbug/ayang1/raw_data/lucas/s1_lucas_2018' # path to raw data
labels = pd.read_csv('/scratch/bbug/ayang1/raw_data/lucas/lucas_2018/copernicus_filtered/lucas_2018_filtered.csv') # lucas labels associated with point_id

# dataframes of s1 signals from each point
lucas_large = [
    'S1_point_10days_10m_1Jan-31Dec_Ireland_ratio-db.csv',
    'S1_point_10days_10m_1Jan-31Dec_Italy_ratio-db.csv',
    'S1_point_10days_10m_1Jan-31Dec_Netherlands_ratio-db.csv',
    'S1_point_10days_10m_1Jan-31Dec_Portugal_ratio-db.csv',
    'S1_point_10days_10m_1Jan-31Dec_Belgium_ratio-db.csv',
    'S1_point_10days_10m_1Jan-31Dec_Bulgaria_ratio-db.csv',
    'S1_point_10days_10m_1Jan-31Dec_Hungary_ratio-db.csv',
]


/tmp/ipykernel_2604672/1074388608.py:3: DtypeWarning: Columns (34,39,44) have mixed types. Specify dtype option on import or set low_memory=False.
  labels = pd.read_csv('/scratch/bbug/ayang1/raw_data/lucas/lucas_2018/copernicus_filtered/lucas_2018_filtered.csv') # lucas labels associated with point_id


In [12]:
# Combine datasets and add labels
raw_s1_signals = []
for path in lucas_large:
    raw_s1_signals.append(pd.read_csv(os.path.join(data_root, path)))
raw_s1_signals = pd.concat(raw_s1_signals)

labeled_s1_signals = add_lucas_labels(raw_s1_signals, labels)
labeled_s1_signals = drop_labels(labeled_s1_signals)

column_mask = labeled_s1_signals.columns.str.contains('VHVV')
labeled_s1_signals.drop(columns=labeled_s1_signals.columns[column_mask], inplace=True)

Loading files
Creating dataset of size 369635


100%|██████████| 369635/369635 [02:03<00:00, 3004.68it/s]


Dropped classes: ['B14', 'B17', 'B18', 'B19', 'B23', 'B35', 'B36', 'B37', 'B42', 'B43', 'B44', 'B71', 'B73', 'B74', 'B75', 'B81', 'B82', 'B83']
Dataset Length:  363956


/tmp/ipykernel_2604672/4080168930.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_s1_signals.drop(columns=labeled_s1_signals.columns[column_mask], inplace=True)


In [13]:
bands = ['VH', 'VV']
band_arrs = get_band_arrays(labeled_s1_signals, bands)

In [14]:
data = np.concatenate([arr[:, np.newaxis, :] for arr in band_arrs.values()], axis=1)
labels = labeled_s1_signals['LABEL']
train_signals, test_signals, train_labels, test_labels = train_test_split(data, labels, test_size=0.3, random_state=42)
train_signals, val_signals, train_labels, val_labels = train_test_split(train_signals, train_labels, test_size=0.1, random_state=42)

In [15]:
print(len(train_labels), len(val_labels), len(test_labels))

229292 25477 109187


In [16]:
dataset = {
    'train_signals': train_signals,
    'train_labels': train_labels,
    'val_signals': val_signals,
    'val_labels': val_labels,
    'test_signals': test_signals,
    'test_labels': test_labels
}

for k, v in dataset.items():
    np.save(os.path.join(save_path, k), v, allow_pickle=True)
